# Exploring prediction markets
> I want to explore some ideas about making bets

- toc: true 
- badges: true
- comments: true
- categories: [prediction markets, betting]

# Introduction to ideas

Alice and Bob are rational agents with their own current beliefs about whether Mr. T. will win the election. Mr. T. will either win or not, and so they view it as a Bernoulli variable $X$. Truly $X \sim \text{Ber}(\theta_{true})$.

Alice and Bob both have point estimates about the probability. They are $\theta_{a}$ and $\theta_{b}$ respectively, and therefore their beliefs are represented by their belief distributions $p_{a}(x) = \text{Ber}(x; \theta_{a})$ and $p_{b}(x) = \text{Ber}(x; \theta_{b})$.

How can Alice and Bob set up a bet they are both satisfied with?

# Pot splitting

The first idea Alice and Bob have is the following: They both put 50 cent in a pot, so now there is \\$1 in the pot.

Suppose they know, they will not agree about who wins the election. The outcome $x$ of $X$ will be unamiguously known. How will they split the pot. Their first idea is the following. A gets fraction $f_a$ of the outcome,

$$
f_b(x) = \frac{p_a(x)}{p_a(x) + p_b(x)}
$$

Then *given* that A has belief $p_a$, A should expect to win
$$
\mathbb{E}_{x \sim p_a}[f_a(x)] = \theta_a \frac{\theta_a}{\theta_a + \theta_b} + (1-\theta_a) \frac{1 - \theta_a}{(1-\theta_a) + (1-\theta_b)}
$$

Likewise B expects $\mathbb{E}_{x \sim p_b}[f_b]$. Making a rule that $\theta_a$ and $\theta_b$ should be in the open interval $(0,1)$ - no absolute certainty - is sufficient condition for this expectation being defined, since then there is no division by zero. Also, only a Sith deals in absolutes.

If instead of just two players and two outcomes, there are $M$ players and $K$ outcomes (a categorical variable), then an equivalent rule for how much each player $m$ should get would be
$$
f_m(k) = \frac{p_m(k)}{\sum_{i=1}^{M} p_i(k)}
$$
and player $m$ would expect to win
$$\mathbb{E}_{k \sim p_m}[f_m(k)] = \sum_{k=1}p_m(k) f_m(k) = \sum_{k=1} p_m(k) \frac{p_m(k)}{\sum_{i=1}^{M} p_i(k)}$$

# Would Alice and Bob participate in such a bet?